In [1]:
import numpy as np
import h5py
import tifffile
from scipy.ndimage.filters import median_filter
from skimage.filters import threshold_otsu
from collections import Counter

## Processing of a single image

## Loading the HDF5 file and converting to tiff

In [4]:
f = h5py.File("example_sample.hdf5", 'r')
my_array = f['data'][()]
img = my_array[0, 0, :, :, :].astype(np.float32)
tifffile.imsave("example_converted.tiff",  img)
print (img.shape)

(288, 288, 288)


## Denoising and thresholding

In [5]:
im_in = tifffile.imread("example_converted.tiff")

#apply single pixel denoising
im_in = median_filter(im_in, size=(3, 3, 3))

#cutaway outer noise area
im_in = im_in[40:240, 40:240, 40:240]

#Normalize to range zero and one
im_in = im_in/255.

#Threshhold Image
threshold_global_otsu = threshold_otsu(im_in)
segmented_image = (im_in >= threshold_global_otsu).astype(np.int32)

#Store as postprocessed image
tifffile.imsave("postprocessed_example.tiff", segmented_image.astype(np.int32))

## Compute porosity

In [7]:
segmented_image = tifffile.imread("postprocessed_example.tiff")
porc = Counter(segmented_image.flatten())
print(porc)
porosity = porc[0]/float(porc[0]+porc[1])
print ("Porosity of the sample: %s" %(porosity))

Counter({1: 6425472, 0: 1574528})
Porosity of the sample: 0.196816
